In [ ]:
import pandas as pd
import numpy as np
import glob

import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
df = pd.read_csv("./data/230907_basic_table_for_analysis_cleaned.csv", index_col=0, low_memory=False)

: 

In [ ]:
df.info()


: 

In [ ]:
df['hrmn']

: 

In [ ]:
# cleanning variables that might not be used in the model
df['date']= pd.to_datetime(df.date,format='%Y-%m-%d')
df['year'] = df['date'].dt.year.astype(int)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['actp'] = le.fit_transform(df['actp'].astype(str))
df['hrmn'] = le.fit_transform(df['hrmn'].astype(str))

: 

In [ ]:
df_M = df[df['metropolitan'] == 1]
df_M.head(3)

: 

In [ ]:
#cleanning variables that might not be used in the model
df_rf = df_M.drop(['num_veh','adr','date','holiday','lat', 'long','grav'], axis=1) # grav removed because is similat to target 

: 

In [ ]:
#split the data for modeling
# 2005 to 2019 for trainning  and 2020 to 2021 to test

# Filter data for the training period (2005-2018)
train_data = df_rf[(df_rf['an'] >= 2005) & (df_rf['an'] <= 2019)]

# Filter data for the testing period (2019-2021)
test_data = df_rf[(df_rf['an'] >= 2020) & (df_rf['an'] <= 2021)]
 #Separate features and target variable
X_train = train_data.drop('fatal', axis=1)  # Features for training
y_train = train_data['fatal']  # Target variable for training

# Separate features and target variable
X_test = test_data.drop('fatal', axis=1)  # Features for training
y_test = test_data['fatal']  # Target variable for training


: 

In [ ]:
df_rf['fatal'].value_counts(normalize=True)

: 

In [ ]:
# import classifier
from sklearn.ensemble import RandomForestClassifier

# import metrics
from sklearn.metrics import classification_report, confusion_matrix

# import evaluation tools
from sklearn.model_selection import KFold, cross_val_score

: 

In [ ]:
 #instantiate RandomForestClassifier with entropy and class_weight
forest_1 = RandomForestClassifier(random_state=4, criterion='entropy', n_jobs=-1, class_weight='balanced')

# train
forest_1.fit(X_train, y_train)

# predict
y_test_preds  = forest_1.predict(X_test)

# evaluate
report = classification_report(y_test, y_test_preds)
print('Classification Report Random Forest - with Entropy and class_weight Parameter: \n', report)

: 

In [ ]:
# create confusion matrix# create confusion matrix
matrix = confusion_matrix(y_test, y_test_preds)

# create dataframe
class_names = df_rf.fatal.values
dataframe = pd.DataFrame(matrix, index=['non-fatal', 'fatal'], 
                         columns=['non-fatal', 'fatal'])

# create heatmap
sns.heatmap(dataframe, annot=True, cbar=None, cmap='Blues')
plt.title('Confusion Matrix')
plt.tight_layout(), plt.xlabel('True Values'), plt.ylabel('Predicted Values')
plt.show()



: 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier instance (replace with your hyperparameters)
forest_1 = RandomForestClassifier(random_state=42)

# Perform cross-validation with F1 score
f1_scores = cross_val_score(forest_1, X_train, y_train, scoring='f1_macro', n_jobs=-1)

# Print the F1 scores for each fold
print("F1 Scores:", f1_scores)




: 